In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../..')

In [5]:
from utils.intermediate_layer_extraction import return_layer_input
from utils.ri_calculator import computeRi,computeRiNaif
import deepmatcher as dm
import torch
from tqdm import tqdm
import pandas as pd

In [7]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/beer_hybrid.pth')

In [8]:
negatives,neg_ids = return_layer_input('../../Structured/Beer/','negatives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [9]:
positives,pos_ids = return_layer_input('../../Structured/Beer','positives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [11]:
hybrid_model = hybrid_model.cuda()

## Compare naif approach with gradient approach

### Negatives

In [12]:
def compareMethods(df1,df2,columns):
    best1,best2 = (0,0)
    for i in df1.index:
        for col in columns:
            lval = df1.at[i,col]
            rval = df2.at[i,col]
            if(lval == 0 and rval >0) or(rval <lval and rval >0):
                best2+=1
            elif(rval ==0 and lval >0)or(lval < rval and lval >0):
                best1+=1
    return (best1,best2)

In [22]:
attributes = ['Beer_Name','Brew_Factory_Name','Style','ABV']
attribute_len = int(len(negatives[0][0])/len(attributes))

In [24]:
neg_ri,neg_ri_norms = computeRi(hybrid_model.classifier,attributes,negatives,attribute_len,1)

100%|██████████| 128/128 [00:07<00:00, 17.74it/s]


In [25]:
neg_ri_naif,neg_ri_norms_naif = computeRiNaif(negatives,positives,1,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 128/128 [00:03<00:00, 36.96it/s]


In [26]:
neg_variation_gradient = pd.DataFrame(data= neg_ri_norms,columns=attributes)
neg_variation_naif = pd.DataFrame(data= neg_ri_norms_naif,columns=attributes)

In [27]:
compareMethods(neg_variation_gradient,neg_variation_naif,attributes)

(1119, 253)

### Positives

In [28]:
pos_ri,pos_ri_norms = computeRi(hybrid_model.classifier,attributes,positives,attribute_len,0)

100%|██████████| 68/68 [00:05<00:00, 12.88it/s]


In [29]:
pos_ri_naif,pos_ri_norms_naif = computeRiNaif(positives,negatives,0,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 68/68 [00:08<00:00,  7.71it/s]


In [30]:
pos_perturbations_gradient = pd.DataFrame(data= pos_ri_norms,columns=attributes)
pos_perturbations_naif = pd.DataFrame(data= pos_ri_norms_naif,columns=attributes)

In [31]:
compareMethods(pos_perturbations_gradient,pos_perturbations_naif,attributes)

(209, 39)